In [10]:
import os
from dotenv import load_dotenv, find_dotenv
import boto3
import psycopg2
import pandas as pd

<p align="center" style="font-size:0.85em; text-decoration: underline;">
<b>Chargement des variables d’environnement, configuration AWS et Postgres + téléchargement depuis S3</b>
</p>

<p style="font-size:0.85em;">
Chargement des variables depuis le fichier <code>.env</code> :  

- Clés AWS pour accéder au bucket S3 et télécharger le fichier final (<code>hotels_weather_final_ter.csv</code>).  
- Paramètres Postgres pour préparer la connexion à la base de données.  

Le fichier est ensuite récupéré localement via boto3 pour être exploité.
</p>
 


In [ ]:
# Load env
env_path = find_dotenv()
load_dotenv(dotenv_path=env_path, override=True)

# control
print(".env chargé depuis :", env_path)
print("USERNAME =", os.getenv("USERNAME"))

.env chargé depuis : c:\Users\Floriane\Desktop\projets_Jedha\bloc1\recap\project_Kayak\.env
USERNAME = flodussart


In [ ]:
# AWS credentials 
AWS_ACCESS_KEY_ID = os.getenv('AWS_KEY')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_KEY')

#  Postgres connection params 
DB_USER = os.getenv('USERNAME')
DB_PASS = os.getenv('PASSWORD')
DB_HOST = os.getenv('HOSTNAME')
DB_PORT = 5432
DB_NAME = os.getenv('DBNAME')

#  S3 settings
S3_BUCKET = 'flodussartprojectkayak'
S3_FILE_KEY = 'projectKayack/src/final_ter.csv'
LOCAL_FILE = 'data/hotels_weather_final_ter.csv'

In [ ]:
# S3 download 
# Creates a low-level S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name='eu-north-1'
)
s3.download_file(S3_BUCKET, S3_FILE_KEY, LOCAL_FILE)
print(f"Fichier téléchargé : {LOCAL_FILE}")

Fichier téléchargé : hotels_weather_final_ter.csv


In [14]:
df = pd.read_csv(LOCAL_FILE, encoding='utf-8-sig')
df.head()

,hotel_id,hotel_name,hotel_url,hotel_rating,hotel_description,hotel_latitude,hotel_longitude,city_id,city_name,city_latitude,city_longitude,temp_max,humidity,wind_speed,clouds,pop,good_weather_score
0,1,Aigues Marines,https://www.booking.com/hotel/fr/aigues-marine...,9.5,L’établissement Aigues Marines vous accueille ...,43.559720,4.218698,1,Aigues-Mortes,43.566152,4.19154,19.544286,53.714286,5.901429,29.571429,1.0,6.610714
1,2,Appartements 3 étoiles terrasse ou patio intra...,https://www.booking.com/hotel/fr/appartement-3...,9.6,L’établissement Appartements 3 étoiles terrass...,43.565358,4.192750,1,Aigues-Mortes,43.566152,4.19154,19.544286,53.714286,5.901429,29.571429,1.0,6.610714
2,3,Artemia Aigues-Mortes - Hotel avec piscine,https://www.booking.com/hotel/fr/le-royal-hote...,9.1,Featuring free WiFi and a seasonal outdoor swi...,43.576396,4.197818,1,Aigues-Mortes,43.566152,4.19154,19.544286,53.714286,5.901429,29.571429,1.0,6.610714
3,4,Au Cœur des Remparts,https://www.booking.com/hotel/fr/au-coeur-des-...,9.9,L’hébergement Au Cœur des Remparts se trouve à...,43.565401,4.192973,1,Aigues-Mortes,43.566152,4.19154,19.544286,53.714286,5.901429,29.571429,1.0,6.610714
4,5,Chez Céline et Sébastien,https://www.booking.com/hotel/fr/chez-celine-e...,9.4,L’hébergement Chez Céline et Sébastien se situ...,43.570192,4.195081,1,Aigues-Mortes,43.566152,4.19154,19.544286,53.714286,5.901429,29.571429,1.0,6.610714


<p align="center" style="font-size:0.85em; text-decoration: underline;">
<b>Chargement des données dans PostgreSQL</b>
</p>

<p style="font-size:0.85em;">
Importe du fichier <code>hotels_weather_final_ter.csv</code> dans la base PostgreSQL à l’aide de <code>psycopg2</code>.  
Principales étapes :
</p>

<p style="font-size:0.85em;">

1. Connexion à la base via les identifiants <code>.env</code>
2. Réinitialisation de la table <code>hotels_weather</code> avec <code>TRUNCATE</code>
3. Chargement en masse avec <code>COPY FROM STDIN</code> après ouverture du fichier CSV (UTF-8)
4. Validation et fermeture de la connexion.
</p>




In [ ]:
# PostgreSQL connection (psycopg2)
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASS
)
# Create a cursor for executing SQL commands
cur = conn.cursor()

In [ ]:
# Reset target table before reloading
cur.execute("TRUNCATE TABLE hotels_weather;")
conn.commit()

In [ ]:
# Bulk load data into PostgreSQL
with open(LOCAL_FILE, 'r', encoding='utf-8-sig') as f:
    next(f)
    cur.copy_expert("""
        COPY hotels_weather (
            hotel_id, hotel_name, hotel_url, hotel_rating, hotel_description,
            hotel_latitude, hotel_longitude, city_id, city_name,
            city_latitude, city_longitude, temp_max, humidity,
            wind_speed, clouds, pop, good_weather_score
        )
        FROM STDIN WITH CSV
    """, f)

In [ ]:
# Finalize the load
# Commit the transaction and close cursor/connection.
conn.commit()
cur.close()
conn.close()